In [3]:
import requests
from urllib3.exceptions import HTTPError
from bs4 import BeautifulSoup

import re
from io import StringIO

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, html, dcc, Output, Input, callback, no_update, State, ctx

In [331]:
init_states = pd.read_csv('state.txt',  header=0)

In [333]:
init_states

,STATE,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778
5,8,CO,Colorado,1779779
6,9,CT,Connecticut,1779780
7,10,DE,Delaware,1779781
8,11,DC,District of Columbia,1702382
9,12,FL,Florida,294478


In [332]:
#alaska/DC
states = init_states.drop(index=[1, 8])

In [5]:
states.head()

,STATE,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778
5,8,CO,Colorado,1779779


In [6]:
# try:
#   r = requests.get('https://en.wikipedia.org/wiki/2020_United_States_presidential_election_in_Virginia')
#   r.raise_for_status()
# except HTTPError as e:
#   print(f'HTTPError: {e}')
# soup = BeautifulSoup(r.text, 'html.parser')
# table = soup.select('div:has(>#By_city\/county) ~ table')
# table

In [7]:
states['STATE_NAME'].str.replace(' ', '_')

0            Alabama
2            Arizona
3           Arkansas
4         California
5           Colorado
6        Connecticut
7           Delaware
9            Florida
10           Georgia
11            Hawaii
12             Idaho
13          Illinois
14           Indiana
15              Iowa
16            Kansas
17          Kentucky
18         Louisiana
19             Maine
20          Maryland
21     Massachusetts
22          Michigan
23         Minnesota
24       Mississippi
25          Missouri
26           Montana
27          Nebraska
28            Nevada
29     New_Hampshire
30        New_Jersey
31        New_Mexico
32          New_York
33    North_Carolina
34      North_Dakota
35              Ohio
36          Oklahoma
37            Oregon
38      Pennsylvania
39      Rhode_Island
40    South_Carolina
41      South_Dakota
42         Tennessee
43             Texas
44              Utah
45           Vermont
46          Virginia
47        Washington
48     West_Virginia
49         Wi

In [8]:
relevant_years = {year: '' for year in range(2000, 2021, 4)}

states_list = []

for year in relevant_years.keys():
  for state, fip in zip(states['STATE_NAME'].str.replace(' ', '_').str.replace('Washington', 'Washington_(state)'),
                        states['STATE']):
    try:
      r = requests.get(f'https://en.wikipedia.org/wiki/{year}_United_States_presidential_election_in_{state}')
      r.raise_for_status()
    except HTTPError as e:
      print(f'HTTPError: {e}')
    print(f'The {year} election in {state}')
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.select('div:has(>#By_county) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#Results_by_county) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#Results_by_County) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#Results_by_parish) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#By_parish) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#By_county_or_independent_city) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#By_city\/county) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#By_county\/city) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#Results_by_city_and_county) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#Results_by_county_or_independent_city) ~ table')
    if len(table) == 0:
      table = soup.select('div:has(>#County_results) ~ table')
    df = pd.read_html(StringIO(str(table)))[0]
    df.loc[:, 'Year'] = year
    df.loc[:, 'State'] = state
    df.loc[:, 'FIPS'] = fip
    states_list.append(df[:-1])

The 2000 election in Alabama
The 2000 election in Arizona
The 2000 election in Arkansas
The 2000 election in California
The 2000 election in Colorado
The 2000 election in Connecticut
The 2000 election in Delaware
The 2000 election in Florida
The 2000 election in Georgia
The 2000 election in Hawaii
The 2000 election in Idaho
The 2000 election in Illinois
The 2000 election in Indiana
The 2000 election in Iowa
The 2000 election in Kansas
The 2000 election in Kentucky
The 2000 election in Louisiana
The 2000 election in Maine
The 2000 election in Maryland
The 2000 election in Massachusetts
The 2000 election in Michigan
The 2000 election in Minnesota
The 2000 election in Mississippi
The 2000 election in Missouri
The 2000 election in Montana
The 2000 election in Nebraska
The 2000 election in Nevada
The 2000 election in New_Hampshire
The 2000 election in New_Jersey
The 2000 election in New_Mexico
The 2000 election in New_York
The 2000 election in North_Carolina
The 2000 election in North_Dakot

#### Regather mucked data

In [9]:
states.iloc[9]

STATE              15
STUSAB             HI
STATE_NAME     Hawaii
STATENS       1779782
Name: 11, dtype: object

In [10]:
gather = [9, 140, 154, 170, 193, 210, 228, 262]
gather_year = [(i // 49)*4 + 2000 for i in gather]
print(gather_year)

[2000, 2008, 2012, 2012, 2012, 2016, 2016, 2020]


In [11]:
states.iloc[[i % 49 for i in gather]]['STATE']

11    15
44    49
9     12
25    29
48    54
16    20
34    38
19    23
Name: STATE, dtype: int64

In [12]:
to_be_added = []
for state, fip, year in zip(states.iloc[[i % 49 for i in gather]]['STATE_NAME'],
                            states.iloc[[i % 49 for i in gather]]['STATE'],
                            gather_year):
  try:
    r = requests.get(f'https://en.wikipedia.org/wiki/{year}_United_States_presidential_election_in_{state}')
    r.raise_for_status()
  except HTTPError as e:
    print(f'HTTPError: {e}')
  print(f'The {year} election in {state}')
  soup = BeautifulSoup(r.text, 'html.parser')
  table = soup.select('div:has(>#By_county) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#Results_by_county) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#Results_by_County) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#Results_by_parish) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#By_parish) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#By_county_or_independent_city) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#By_city\/county) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#By_county\/city) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#Results_by_city_and_county) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#Results_by_county_or_independent_city) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#County_results) ~ table.wikitable')
  df = pd.read_html(StringIO(str(table)))[0]
  df.loc[:, 'Year'] = year
  df.loc[:, 'State'] = state
  df.loc[:, 'FIPS'] = fip
  to_be_added.append(df[:-1])

The 2000 election in Hawaii
The 2008 election in Utah
The 2012 election in Florida
The 2012 election in Missouri
The 2012 election in West Virginia
The 2016 election in Kansas
The 2016 election in North Dakota
The 2020 election in Maine


In [13]:
to_be_added.pop(6)
to_be_added.pop(4)
to_be_added.pop(3)

,County,Winner,Delegates [20],District,Source,Year,State,FIPS
0,Adair,Romney,9,6th,[21],2012,Missouri,29
1,Andrew,Santorum,8,6th,NaN,2012,Missouri,29
2,Atchison,Santorum,3,6th,NaN,2012,Missouri,29
3,Audrain,Santorum,7,4th,NaN,2012,Missouri,29
4,Barry,Romney,14,7th,[22][23],2012,Missouri,29
...,...,...,...,...,...,...,...,...
109,Warren,Santorum,13,3rd,NaN,2012,Missouri,29
110,Washington,Santorum,7,8th,NaN,2012,Missouri,29
111,Wayne,Santorum,6,8th,NaN,2012,Missouri,29
112,Webster,Santorum,11+4,"4th, 7th",NaN,2012,Missouri,29


In [14]:
gather_last_3 = [170, 193, 228]
last_3 = []
for state, fip, year in zip(states.iloc[[i % 49 for i in gather_last_3]]['STATE_NAME'],
                            states.iloc[[i % 49 for i in gather_last_3]]['STATE'],
                            [2012, 2012, 2016]):
  try:
    r = requests.get(f'https://en.wikipedia.org/wiki/{year}_United_States_presidential_election_in_{state}')
    r.raise_for_status()
  except HTTPError as e:
    print(f'HTTPError: {e}')
  print(f'The {year} election in {state}')
  soup = BeautifulSoup(r.text, 'html.parser')
  table = soup.select('div:has(>h3#Results_by_county) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>#County_results) ~ div > table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>h3#Results_by_county_2) ~ table.wikitable')
  df = pd.read_html(StringIO(str(table)))[0]
  df.loc[:, 'Year'] = year
  df.loc[:, 'State'] = state
  df.loc[:, 'FIPS'] = fip
  last_3.append(df[:-1])

The 2012 election in Missouri
The 2012 election in West Virginia
The 2016 election in North Dakota


#### Gather Alaska and DC Data

In [385]:
dc_years = []
for year in relevant_years.keys():
  try:
    r = requests.get(f'https://en.wikipedia.org/wiki/{year}_United_States_presidential_election_in_the_District_of_Columbia')
    r.raise_for_status()
  except HTTPError as e:
    print(f'HTTPError: {e}')
  print(f'The {year} election in DC')
  soup = BeautifulSoup(r.text, 'html.parser')
  table = soup.select('div:has(>h3#Results) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>h2#Results) ~ table.wikitable')
  if len(table) == 0:
    table = soup.select('div:has(>h3#Results_2) ~ table.wikitable')
  df = pd.read_html(StringIO(str(table)))[0]
  df = df.T
  df.loc[:, 'Year'] = year
  df.loc[:, 'State'] = 'DC'
  df.loc[:, 'FIPS'] = fip
  dc_years.append(df[:-1])

The 2000 election in DC
The 2004 election in DC
The 2008 election in DC
The 2012 election in DC
The 2016 election in DC
The 2020 election in DC


In [393]:
states_df_columns = states_df.columns

In [495]:
states_df_columns

MultiIndex([(    'County',  ''),
            ('Republican', '#'),
            ('Republican', '%'),
            ('Democratic', '#'),
            ('Democratic', '%'),
            (     'Total',  ''),
            (      'Year',  ''),
            (     'State',  ''),
            (      'FIPS',  ''),
            (        'ID',  '')],
           )

In [431]:
index_4th = dc_years[4].index.values
index_4th[5] = 'Percentage'
dc_years[4].index = index_4th

In [444]:
dc_years[5]

,0,1,2,3,4,5,6,7,Year,State,FIPS
Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total votes,2020,DC,2
Party.1,Democratic,Republican,Write-in,Libertarian,DC Statehood Green,Independent,Independent,Total votes,2020,DC,2
Candidate,Joe Biden Kamala Harris,Donald Trump Mike Pence,Write-in,Jo Jorgensen Spike Cohen,Howie Hawkins Angela Walker,Gloria La Riva Sunil Freeman,Brock Pierce Karla Ballard,Total votes,2020,DC,2
Votes,317323,18586,3137,2036,1726,855,693,344356,2020,DC,2
%,92.15,5.4,0.91,0.59,0.5,0.25,0.2,100.0,2020,DC,2


In [504]:
DC_clean = []
for df, index, year in zip(dc_years, [6, 6, 5, 5, 5, 7], [2000, 2004, 2008, 2012, 2016, 2020]):
  try:
    dem_pct = df.loc['Percentage', 0]
    rep_pct = df.loc['Percentage', 1]
  except:
    dem_pct = df.loc['%', 0]
    rep_pct = df.loc['%', 1]
  try:
    dem_totals = df.loc['Votes', 0]
    rep_totals = df.loc['Votes', 1]
    total_votes = df.loc['Votes', index]
  except:
    dem_totals = df.loc['Popular vote', 0]
    rep_totals = df.loc['Popular vote', 1]
    total_votes = df.loc['Popular vote', index]

  # if type(dem_pct) == str:
  #   dem_pct = float(dem_pct.replace('%', ''))
  #   rep_pct = float(rep_pct.replace('%', ''))

  clean_year = pd.Series(['District of Columbia',
             rep_totals,
             rep_pct,
             dem_totals,
             dem_pct,
             total_votes,
             year,
             'DC',
             '11',
             '11001'], index=states_df_columns)
  DC_clean.append(clean_year)

In [505]:
DC_df = pd.DataFrame(DC_clean)

In [506]:
DC_df

County Republican        Democratic           Total  Year  \
                                 #      %          #       %                 
0  District of Columbia      18073  8.95%     171923  85.16%  201894  2000   
1  District of Columbia      21256  9.34%     202970  89.18%  227586  2004   
2  District of Columbia      17367  6.53%     245800  92.46%  265853  2008   
3  District of Columbia      21381  7.28%     267070  90.91%  293764  2012   
4  District of Columbia      12723  4.09%     282830  90.86%  311268  2016   
5  District of Columbia      18586    5.4     317323   92.15  344356  2020   

  State FIPS     ID  
                     
0    DC   11  11001  
1    DC   11  11001  
2    DC   11  11001  
3    DC   11  11001  
4    DC   11  11001  
5    DC   11  11001

#### Prep Gathered Data For Concat

In [507]:
# need to implement after
states_list_copy = [state.copy() for state in states_list]

In [508]:
for i in [262, 228, 210, 193, 170, 154, 140, 9]:
  states_list_copy.pop(i)

In [509]:
states_list_copy = states_list_copy + to_be_added + last_3 + [DC_df]

In [510]:
import re

In [511]:
# 2000 to 2020
parties = [
  'Democratic',
  'Republican',
  'Constitution',
  'Green',
  'Independent',
  'Libertarian',
  'Reform',
  'DFL',
  'NPL',
  'Democratic-NPL'
]

In [512]:
for state_year_df in states_list_copy:
  for party in parties:
    adjusted_cols = state_year_df.columns.get_level_values(0).str.replace(f'.*{party}', party, regex=True)
    level_2 = state_year_df.columns.get_level_values(1)
    multiindex = pd.MultiIndex.from_tuples(zip(adjusted_cols, level_2))
    state_year_df.columns = multiindex

In [513]:
for i, state in enumerate(states_list_copy):
  try:
    state.rename(columns=lambda x: re.sub(r'.*Total votes cast.*', 'Total', x), level=0, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*County.*', 'County', x), level=0, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*Parish.*', 'County', x), level=0, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*NPL.*', 'Democratic', x), level=0, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*DFL.*', 'Democratic', x), level=0, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*Total votes cast.*', '', x), level=1, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*Total.*', '', x), level=1, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*County.*', '', x), level=1, inplace=True)
    state.rename(columns=lambda x: re.sub(r'.*Parish.*', '', x), level=1, inplace=True)
  except:
    print('index', i)

In [514]:
cols_to_keep = [( 'County',	''),	
( 'Republican',	'#'),	
( 'Republican',	'%'),	
( 'Democratic',	'#'),	
( 'Democratic',	'%'),	
( 'Total',	''),	
( 'Year',	''),	
( 'State',	''),	
( 'FIPS',	'')]

In [515]:
prepped_to_concat = []
for i, df in enumerate(states_list_copy):
  try:
    prepped_to_concat.append(df[cols_to_keep])
  except:
    print(i)

states_df = pd.concat(prepped_to_concat)
states_df = states_df.iloc[:, ~states_df.columns.get_level_values(0).str.contains('Margin')]
states_df.loc[:, states_df.columns.get_level_values(1).str.contains('%')] =\
  states_df.loc[:, states_df.columns.get_level_values(1).str.contains('%')].apply(lambda x: x.str.rstrip('%'), axis=0).astype(float)
states_df = states_df.replace(np.nan, 0)
states_df['FIPS'] = states_df['FIPS'].apply(lambda x: str(x).zfill(2))
states_df.reset_index(drop=True, inplace=True)

C:\Users\Akai\AppData\Local\Temp\ipykernel_31380\1373844205.py:12: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [516]:
states_df['County'] = states_df['County'].str.replace('mckean', 'McKean', flags=re.IGNORECASE)

In [4]:
try:
  r = requests.get('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
  r.raise_for_status()
except HTTPError as e:
  print(f'Ran into HTTPError: {e}')
counties = r.json()

In [5]:
counties_df = pd.DataFrame([lo['properties'] for lo in counties['features']])
counties_df['ID'] = counties_df['STATE'] + counties_df['COUNTY']
counties_df

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,ID
0,0500000US01001,01,001,Autauga,County,594.436,01001
1,0500000US01009,01,009,Blount,County,644.776,01009
2,0500000US01017,01,017,Chambers,County,596.531,01017
3,0500000US01021,01,021,Chilton,County,692.854,01021
4,0500000US01033,01,033,Colbert,County,592.619,01033
...,...,...,...,...,...,...,...
3216,0500000US51001,51,001,Accomack,County,449.496,51001
3217,0500000US51021,51,021,Bland,County,357.725,51021
3218,0500000US51027,51,027,Buchanan,County,502.763,51027
3219,0500000US51037,51,037,Charlotte,County,475.271,51037


#### Associate GEOjson IDs 

In [27]:
counties_df[counties_df['LSAD'] == 'city']

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,ID
597,0500000US51510,51,510,Alexandria,city,15.027,51510
598,0500000US51530,51,530,Buena Vista,city,6.704,51530
599,0500000US51540,51,540,Charlottesville,city,10.238,51540
600,0500000US51595,51,595,Emporia,city,6.894,51595
601,0500000US51660,51,660,Harrisonburg,city,17.418,51660
602,0500000US51670,51,670,Hopewell,city,10.278,51670
603,0500000US51760,51,760,Richmond,city,59.805,51760
604,0500000US51775,51,775,Salem,city,14.437,51775
605,0500000US51820,51,820,Waynesboro,city,15.039,51820
1549,0500000US29510,29,510,St. Louis,city,61.909,29510


In [28]:
fips_plus_county_to_index = {v.lower().replace(' ', ''): k for k, v in (counties_df['STATE'] + counties_df['NAME']).items()}

fips_plus_county_to_index_w_city = {v.lower().replace(' ', ''): k for k, v in (counties_df['STATE'] + counties_df['NAME'] +\
                            ((counties_df['LSAD'] == 'city').apply(lambda x: 'city' if x else ''))).items()}
index_to_id = {k: v for k, v in counties_df['ID'].items()}
# index_to_id_w_city = {v: k for k, v in fips_plus_county_to_index_w_city.items()}

In [29]:
without_city = (states_df['FIPS'] + states_df['County']).str.lower().str.replace(' ', '').map(fips_plus_county_to_index).map(index_to_id)
with_city = (states_df['FIPS'] + states_df['County']).str.lower().str.replace(' ', '').map(fips_plus_county_to_index_w_city).map(index_to_id)

NameError: name 'states_df' is not defined

In [ ]:
ids = with_city.combine_first(without_city)

NameError: name 'with_city' is not defined

In [ ]:
states_df['ID'] = ids

NameError: name 'ids' is not defined

In [ ]:
null_ids = ids[ids.isnull()].index.tolist()

NameError: name 'ids' is not defined

In [ ]:
len(null_ids)

NameError: name 'null_ids' is not defined

In [ ]:
with pd.option_context('display.max_rows', None):
  display(states_df.iloc[null_ids]['County'])

NameError: name 'pd' is not defined

In [30]:
states_df.iloc[14404]

NameError: name 'states_df' is not defined

In [31]:
states_df.loc[states_df['County'] == 'Coös', 'ID'] = '33007'
states_df.loc[states_df['County'] == 'Dona Ana', 'ID'] = '35013'
states_df.loc[states_df['County'] == 'Oglala Lakota', 'ID'] = '46113'

NameError: name 'states_df' is not defined

In [32]:
states_df[states_df['County'] == 'Oglala Lakota']['ID']

NameError: name 'states_df' is not defined

In [33]:
states_df.loc[states_df['ID'] == '46113', 'County'] = 'Oglala Lakota'

NameError: name 'states_df' is not defined

In [34]:
states_df['State'] = states_df['State'].str.replace('Washington_(state)', 'Washington').str.replace('_', ' ')
states_df['County'] = states_df['County'].str.replace('Coös', 'Coos')\
                                        .str.replace('Desoto', 'DeSoto')\
                                        .str.replace('Doña Ana', 'Dona Ana')\
                                        .str.replace('Le Flore', 'LeFlore')

NameError: name 'states_df' is not defined

In [35]:
states_df['State'] = states_df['State'].str.replace('Washington_(state)', 'Washington').str.replace('_', ' ')
states_df['County'] = states_df['County'].str.replace('Coös', 'Coos')\
                                        .str.replace('Desoto', 'DeSoto')\
                                        .str.replace('Doña Ana', 'Dona Ana')\
                                        .str.replace('Le Flore', 'LeFlore')
states_df['Total'] = states_df['Total']\
  .str.replace(r'\[\w+\]', '', regex=True)\
  .str.replace(',', '').astype(int)
states_df[states_df.isnull()['ID']]

# Combine Clifton Forge numbers with Alleghany in 2000
Alleghany2k = states_df[(states_df['County'] == 'Alleghany') & (states_df['State'] == 'Virginia') & (states_df['Year'] == 2000)]
Clifton_Forge2k = states_df[(states_df['County'] == 'Clifton Forge')]
new_Alleghany2k = Alleghany2k[[('Democratic', '#'), ('Republican', '#'), ('Total', '')]].values + Clifton_Forge2k[[('Democratic', '#'), ('Republican', '#'), ('Total', '')]].values
states_df.loc[2787, ('Republican', '#')] = new_Alleghany2k[0][0]
states_df.loc[2787, ('Democratic', '#')] = new_Alleghany2k[0][1]
states_df.loc[2787, ('Total', '')] = new_Alleghany2k[0][2]
states_df.loc[2787, ('Republican', '%')] = round(new_Alleghany2k[0][0] / new_Alleghany2k[0][2] * 100, 2)
states_df.loc[2787, ('Democratic', '%')] = round(new_Alleghany2k[0][1] / new_Alleghany2k[0][2] * 100, 2)
states_df = states_df.dropna(axis=0).reset_index(drop=True)

NameError: name 'states_df' is not defined

In [36]:
with pd.option_context('display.max_rows', None):
  display(counties_df[counties_df['STATE'] == '51'].sort_values(by='NAME'))

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,ID
3216,0500000US51001,51,001,Accomack,County,449.496,51001
1899,0500000US51003,51,003,Albemarle,County,720.698,51003
597,0500000US51510,51,510,Alexandria,city,15.027,51510
1800,0500000US51005,51,005,Alleghany,County,445.457,51005
650,0500000US51007,51,007,Amelia,County,355.269,51007
1900,0500000US51009,51,009,Amherst,County,473.934,51009
1801,0500000US51011,51,011,Appomattox,County,333.494,51011
1802,0500000US51013,51,013,Arlington,County,25.974,51013
1803,0500000US51015,51,015,Augusta,County,966.999,51015
651,0500000US51017,51,017,Bath,County,529.158,51017


#### Prep Map Data

In [6]:
import_cols = [( 'County',	''),	
  ( 'Republican',	'#'),	
  ( 'Republican',	'%'),	
  ( 'Democratic',	'#'),	
  ( 'Democratic',	'%'),	
  ( 'Total',	''),	
  ( 'Year',	''),	
  ( 'State',	''),	
  ( 'FIPS',	''),
  ( 'ID',	''),
]
states_df = pd.read_csv('counties_minus_alaskan.csv', index_col=0, skiprows=2, names=import_cols, dtype={('FIPS',''): object,('ID',''): object})
states_df

County Republican        Democratic          Total  Year  \
                                     #      %          #      %                 
0                   Autauga      11993  69.69       4942  28.72   17208  2000   
1                   Baldwin      40872  72.37      13997  24.78   56480  2000   
2                   Barbour       5096  49.02       5188  49.91   10395  2000   
3                      Bibb       4273  60.17       2710  38.16    7101  2000   
4                    Blount      12667  70.48       4977  35.67   17973  2000   
...                     ...        ...    ...        ...    ...     ...   ...   
18675  District of Columbia      21256   9.34     202970  89.18  227586  2004   
18676  District of Columbia      17367   6.53     245800  92.46  265853  2008   
18677  District of Columbia      21381   7.28     267070  90.91  293764  2012   
18678  District of Columbia      12723   4.09     282830  90.86  311268  2016   
18679  District of Columbia      18586   5.40     317323  92.15  344356  2020   

         State FIPS     ID  
                            
0      Alabama   01  01001  
1      Alabama   01  01003  
2      Alabama   01  01005  
3      Alabama   01  01007  
4      Alabama   01  01009  
...        ...  ...    ...  
18675       DC   11  11001  
18676       DC   11  11001  
18677       DC   11  11001  
18678       DC   11  11001  
18679       DC   11  11001  

[18680 rows x 10 columns]

In [7]:
counties_rename_df = counties_df.rename({'NAME': 'County', 'STATE': 'FIPS'})

In [8]:
counties_geometry = pd.DataFrame({'id': lo['id'], 'coordinates': lo['geometry']['coordinates']} for lo in counties['features'])
counties_geometry = counties_geometry.set_index('id')
counties_geometry.index.name = None

In [9]:
quant_cols = [
('Republican', '#'),
('Republican', '%'),
('Democratic', '#'),
('Democratic', '%'),
( 'Total', '')]

In [10]:
grouped = states_df.groupby(by=['ID'])[[('Democratic', '%'), ('Republican', '%')]].std().sort_values(('Democratic', '%')).sort_values(by=('Democratic', '%'), ascending=False)

In [11]:
diff_v_prev_year = states_df.groupby(by='ID')[quant_cols].diff()

In [12]:
states_df

County Republican        Democratic          Total  Year  \
                                     #      %          #      %                 
0                   Autauga      11993  69.69       4942  28.72   17208  2000   
1                   Baldwin      40872  72.37      13997  24.78   56480  2000   
2                   Barbour       5096  49.02       5188  49.91   10395  2000   
3                      Bibb       4273  60.17       2710  38.16    7101  2000   
4                    Blount      12667  70.48       4977  35.67   17973  2000   
...                     ...        ...    ...        ...    ...     ...   ...   
18675  District of Columbia      21256   9.34     202970  89.18  227586  2004   
18676  District of Columbia      17367   6.53     245800  92.46  265853  2008   
18677  District of Columbia      21381   7.28     267070  90.91  293764  2012   
18678  District of Columbia      12723   4.09     282830  90.86  311268  2016   
18679  District of Columbia      18586   5.40     317323  92.15  344356  2020   

         State FIPS     ID  
                            
0      Alabama   01  01001  
1      Alabama   01  01003  
2      Alabama   01  01005  
3      Alabama   01  01007  
4      Alabama   01  01009  
...        ...  ...    ...  
18675       DC   11  11001  
18676       DC   11  11001  
18677       DC   11  11001  
18678       DC   11  11001  
18679       DC   11  11001  

[18680 rows x 10 columns]

In [13]:
grouped

,Democratic,Republican
,%,%
ID,,
21119,20.890754,20.429151
54059,20.340503,20.002455
54047,19.943606,19.673880
21063,19.866033,19.392181
47061,18.900512,18.657995
...,...,...
40139,1.183785,2.218318
28059,1.176566,0.980668


In [14]:
grouped = grouped.reset_index()

#### Build Map

In [15]:
states_df = states_df.sort_values(by='Year').reset_index(drop=True)

In [16]:
app = Dash(__name__)

df = states_df[states_df['Year'] == 2020]

prev_location= ['']
cur_location = ['']

app.layout = html.Div([
  dcc.Dropdown(['Election Outcomes', 'Election Swing'], 'Election Outcomes', id='mode-select'),
  html.Div([
    html.Div([
      dcc.Graph(
        id='election-map', clear_on_unhover=True
      ),
    ], id='election-map-box', style={'width': '100%'}),
    html.Div([
        html.Div([
          html.H4(id='location-name', style={'display': 'inline-block', 'margin-left': '10px'}),
          html.Button(id='county-info-box-button',
                      children='Close Graph',
                      style={'display': 'inline-block',
                             'margin-right': '10px'}
                      )
          ],
          style={'height': '45px',
                 'display': 'flex',
                 'justify-content': 'space-between',
                 'background': 'white',
                 'text-align': 'center',
                 'align-items': 'center'}
        ),
        html.Div([
          dcc.Graph(
            id='county-info',
          )],
        )
    ], id='county-info-box', style={'width': '0', 'display': None})
  ]),
  dcc.Tooltip(id='election-hover-results'),
  dcc.Slider(
    states_df['Year'].min(),
    states_df['Year'].max(),
    step=4,
    id='election-map-slider',
    marks={str(year): str(year) for year in states_df['Year'].unique()},
    value=2000,
    included=False,
  ),
])


@callback(
    Output('election-map-box', 'children'),
    Input('mode-select', 'value')
)
def select_map(value):
  if value == 'Election Outcomes':
    return dcc.Graph(id='election-map', clear_on_unhover=True),
  elif value == 'Election Swing':
    return dcc.Graph(id='swing-map', clear_on_unhover=True),

@callback(
    Output('swing-map', 'figure'),
    Input('mode-select', 'value')
)
def set_swing_map(value):
  fig = go.Figure(go.Choroplethmapbox(
                                      geojson=counties,
                                      locations=grouped['ID'],
                                      z=grouped[('Democratic', '%')],
                                      colorscale='BuPu',
                                      zmin=0,
                                      zmax=25,
                                      marker_opacity=0.6,
                                      marker_line_width=0,
                                      name='',
                                      showscale=False
  ))
  fig['layout']['uirevision'] = True
  fig.update_layout(mapbox_style='carto-positron',
                    mapbox_zoom=3,
                    mapbox_center={'lat': 37.0902, 'lon': -95.7129})
  fig.update_layout(margin={'r': 0, 't': 0, 'b': 0, 'l': 0})
  return fig

@callback(
    Output('election-hover-results', 'show'),
    Output('election-hover-results', 'bbox'),
    Output('election-hover-results', 'children'),
    Input('election-map', 'hoverData'),
    Input('county-info-box', 'style')
)
def display_hover_results(hover_data, style):
  if hover_data is None or style['display'] == 'inline-block':
    return False, no_update, no_update
  customdata = hover_data['points'][0]['customdata']
  bbox = hover_data['points'][0]['bbox']

  children = [
    html.Div([
      html.B(f'{customdata[1]} County'),
      html.Br(),
      html.P('Parties', style={'fontSize':'12px', 'opacity': .8}),
      html.Hr(),
      html.Div([
        html.P(f'Democratic: {customdata[2]}%'),
        html.P(f'Republican: {customdata[3]}%')
      ])
    ])
  ]
  return True, bbox, children

@callback(
  Output('election-map', 'figure'),
  Output('election-map-box', 'style'),
  Output('county-info-box', 'style'),
  Output('county-info', 'figure'),
  Output('location-name', 'children'),
  Output('election-map', 'clickData'),
  Input('election-map-slider', 'value'),
  Input('election-map', 'clickData'),
  Input('county-info-box-button', 'n_clicks'),
)
def update_map(value, click_data, _):
  cdf = states_df[(states_df['Year']) == int(value)]
  dem_and_repub_pct = pd.concat([cdf['ID'], cdf[('County', '')], cdf[('Democratic', '%')], cdf[('Republican', '%')]], axis=1)
  fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                                      customdata=dem_and_repub_pct,
                                      locations=cdf['ID'],
                                      z=cdf[('Democratic', '%')],
                                      colorscale='Bluered_r',
                                      zmin=0,
                                      zmax=100,
                                      marker_opacity=.6,
                                      marker_line_width=0,
                                      name='',
                                      showscale=False
                                      ))

  fig['layout']['uirevision'] = True
  fig.update_layout(mapbox_style='carto-positron',
                    mapbox_zoom=3,
                    mapbox_center={'lat': 37.0902, 'lon': -95.7129})
  fig.update_layout(margin={'r': 0, 't': 0, 'b': 0, 'l': 0})
  
  election_map_box_style = no_update
  county_info_box_style = no_update
  info_fig = no_update
  location_name = no_update
  
  if click_data is not None:
    prev_location[0] = cur_location[0]
    cur_location[0] = click_data['points'][0]['location']

  if prev_location[0] != cur_location[0] and ctx.triggered_id != 'county-info-box-button':
    #consider moving this process to the geometry df
    coords = counties_geometry.loc[cur_location[0], 'coordinates'][0]
    lon, lat = zip(*coords)
    lon += (lon[0], )
    lat += (lat[0], )
    fig.add_trace(go.Scattermapbox(
      mode='lines',
      lat=lat,
      lon=lon,
      line={'width': 1, 'color': 'black'},
      name=cur_location[0],
    ))

    election_map_box_style = {'display': 'inline-block', 'width': '60%'}
    county_info_box_style = {'display': 'inline-block', 'width': '40%'}


    x_max = max(lat)
    x_min = min(lat)
    y_max = max(lon)
    y_min = min(lon)
    x_diff = abs(x_max - x_min)
    y_diff = abs(y_max - y_min)
    max_diff = max(x_diff, y_diff)
    zoom = 11.5 - np.log(max_diff * 111)
    fig.update_layout(mapbox_zoom=zoom,
                      mapbox_center={'lat': x_min + (x_diff / 2), 'lon': y_min + (y_diff / 2)})

    location_df = states_df[states_df['ID'] == cur_location[0]]
    county = location_df['County'].values[0]
    location_name = f'{county} County'
    third_party = 100 - (location_df[('Democratic', '%')] + location_df[('Republican', '%')])
    info_fig = go.Figure()
    info_fig.add_trace(go.Scatter(
      x=location_df['Year'].sort_values(),
      y=location_df[('Democratic', '%')],
      connectgaps=True,
      name='Democratic'
      )
    )
    info_fig.add_trace(go.Scatter(
      x=location_df['Year'].sort_values(),
      y=location_df[('Republican', '%')],
      connectgaps=True,
      name='Republican'
      )
    )
    info_fig.add_trace(go.Scatter(
      x=location_df['Year'].sort_values(),
      y=third_party,
      connectgaps=True,
      name='Other'
      )
    )

    info_fig.update_layout(hovermode='x unified',
                          showlegend=False,
                          margin={'r': 0, 't': 20, 'b': 0, 'l': 5},
                          yaxis_range=[0, 100],
                          height=405)
    info_fig.update_xaxes(tickvals=location_df['Year'].sort_values())
    info_fig.update_yaxes(tickvals=list(range(0, 101, 20)))

  else:
    prev_location[0] = ''
    cur_location[0] = ''
    election_map_box_style = {'display': 'inline-block', 'width': '100%'}
    county_info_box_style = {'display': 'none', 'width': '0'}

  # if ctx.triggered_id == 'county-info-box-button':
  #   election_map_box_style = {'display': 'inline-block', 'width': '100%'}
  #   county_info_box_style = {'display': 'inline-block', 'width': '0'}

  fig.update_traces(hoverinfo="none", hovertemplate=None)

  return fig, election_map_box_style, county_info_box_style, info_fig, location_name, None

app.run(debug=True)


In [39]:
states_df.sort_values(by='Year').reset_index(drop=True)

County Republican        Democratic          Total  Year  \
                                     #      %          #      %                 
0                   Autauga      11993  69.69       4942  28.72   17208  2000   
1                  Paulding       5210  58.24       3384  37.83    8946  2000   
2                     Perry       6440  50.20       5895  45.95   12828  2000   
3                  Pickaway      10717  60.41       6598  37.19   17740  2000   
4                      Pike       5333  50.50       4923  46.62   10560  2000   
...                     ...        ...    ...        ...    ...     ...   ...   
18675                 Alger       3014  58.70       2053  39.98    5135  2020   
18676               Allegan      41392  61.60      24449  36.39   67195  2020   
18677                Alpena      10686  62.91       6000  35.32   16987  2020   
18678               Hampden      87318  40.02     125948  57.73  218177  2020   
18679  District of Columbia      18586   5.40     317323  92.15  344356  2020   

               State FIPS     ID  
                                  
0            Alabama   01  01001  
1               Ohio   39  39125  
2               Ohio   39  39127  
3               Ohio   39  39129  
4               Ohio   39  39131  
...              ...  ...    ...  
18675       Michigan   26  26003  
18676       Michigan   26  26005  
18677       Michigan   26  26007  
18678  Massachusetts   25  25013  
18679             DC   11  11001  

[18680 rows x 10 columns]

In [48]:
states_df[states_df['ID'] == '06075']

County Republican        Democratic          Total  Year  \
                              #      %          #      %                 
2285   San Francisco      51496  16.10     241578  75.54  319786  2000   
5251   San Francisco      54355  15.21     296772  83.02  357465  2004   
8489   San Francisco      52292  13.62     322220  83.96  383796  2008   
11429  San Francisco      47076  13.01     301723  83.40  361795  2012   
14626  San Francisco      37688   9.23     345084  84.47  408541  2016   
17406  San Francisco      56417  12.72     378156  85.26  443553  2020   

            State FIPS     ID  
                               
2285   California   06  06075  
5251   California   06  06075  
8489   California   06  06075  
11429  California   06  06075  
14626  California   06  06075  
17406  California   06  06075